## **0. Introduction**

In recently year, the development of science and technology has had many outstanding achievements in general and in the field of Artificial Intelligence (AI) field in particular. AI almost become a effective supporter for human in many fields, such as language, sound, image,... Especially, 

In this projects, we will build small application about vehicle license plate recegnition

In [19]:
!nvidia-smi

Thu Jan 16 02:17:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX230           Off | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              N/A / ERR! |      6MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0


## **1. Libraries**

In [69]:
import requests
import os
import warnings
import zipfile
import pandas as pd
import cv2
import tensorflow as tf

from tqdm import tqdm
from lxml import etree

In [37]:
warnings.filterwarnings('ignore')

In [23]:
# Check GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [24]:
# Enable all GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

## **2. Dataset**

...

In [82]:
def download_dataset(url, dest_path = 'data'):
    # Create folder data to store dataset if it does not exist
    if not os.path.exists(dest_path):
        print('Creating folder to store dataset...')
        os.mkdir(dest_path)
        print('Created!!')
    
    # File name and destination path where file is stored
    zip_file_name = url.split('/')[-1]
    zip_file_path = f'{dest_path}/{zip_file_name}'

    # Download dataset if data folder is empty
    if len(os.listdir(dest_path)) == 0:
        print('Dataset does not exist, please waiting to download data from github...')
        # Download zip file
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('Content-Length', 0))
        print('Start to download...')
        
        # Display progress when download file
        with open(zip_file_path, 'wb') as file, tqdm(
            desc=f'{zip_file_path}',
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
            leave=True,
            position=0,
        ) as progress_bar:
            for content in response.iter_content(chunk_size=1024):
                size = file.write(content)
                progress_bar.update(size)
        print('Downloaded, please waiting to extract file...')
        print('Extracting...')

        # Extract zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(dest_path)
            print('Extracted!!')
        print('Removing zip file...')
        os.remove(zip_file_path)
        print('Removed!!')
    print('Done!!')

In [84]:
url = 'https://github.com/lngquoctrung/vehicle-license-plate-recognition/raw/refs/heads/main/archive.zip'
download_dataset(url=url)

Creating folder to store dataset...
Created!!
Dataset does not exist, please waiting to download data from github...
Start to download...


data/archive.zip: 100%|██████████████████████| 203M/203M [00:13<00:00, 16.1MB/s]


Downloaded, please waiting to extract file...
Extracting...
Extracted!!
Removing zip file...
Removed!!
Done!!


In [130]:
# Check data folder
for dir_name in os.listdir('./data'):
    print(f'{dir_name}:')
    list_child_files = os.listdir(f'./data/{dir_name}')
    for i in range(len(list_child_files[:5])):
        print(f'{' ' * 2}|{'-' * 3} {list_child_files[i]}')
    print(f'{' ' * 2}...')

annotations:
  |--- Cars24.xml
  |--- Cars367.xml
  |--- Cars346.xml
  |--- Cars430.xml
  |--- Cars40.xml
  ...
images:
  |--- Cars354.png
  |--- Cars133.png
  |--- Cars261.png
  |--- Cars328.png
  |--- Cars416.png
  ...
